# EDA on data from gas benchmarks

#### Maria Silva, December 2025

In [91]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [92]:
# plotting theme
sns.set_theme(
    style="whitegrid", palette="Set2", rc={"figure.dpi": 500, "axes.titlesize": 15}
)

## Load and process data

In [93]:
# Main directories
current_path = os.getcwd()
repo_dir = os.path.abspath(os.path.join(current_path, ".."))
data_dir = os.path.join(repo_dir, "data")

In this analysis, we are using data generated by running the [EEST benchmark suite](https://github.com/ethereum/execution-spec-tests/tree/main/tests/benchmark) with the [Nethermind benchmarking tooling](https://github.com/NethermindEth/gas-benchmarks). We extracted this data in 15-12-2025.

In [94]:
df = pd.read_csv(os.path.join(data_dir, "min_mgas_s_by_test_and_client_2025-12-15.csv"))

# The CSV has column names with embedded client_name and test_title
# Format: min_mgas_s {client_name="...", test_title="..."}
# We need to extract these and reshape the data

import re

# Get all columns except "Time" (if it exists)
data_cols = [col for col in df.columns if col != "Time"]

# Extract client_name and test_title from column names
rows = []
for col in data_cols:
    match = re.search(r'client_name="([^"]+)".*test_title="([^"]+)"', col)
    if match:
        client = match.group(1)
        test_title = match.group(2)
        # Get all values from this column
        for idx, value in df[col].items():
            rows.append({
                "test_title": test_title,
                "client": client,
                "mgas/s": value
            })

df = pd.DataFrame(rows)

# Fix rows with "k" and sort
df["mgas/s"] = df["mgas/s"].replace({" K": "e3"}, regex=True).astype(float)
df = df.sort_values(by="mgas/s")

# Parse test title
df["test_file"] = (
    df["test_title"].str.replace("tests_benchmark_", "").str.split(".py").str[0]
)
df["test_name"] = df["test_title"].str.split(".py__").str[1].str.split("[").str[0]
df["test_params"] = (
    df["test_title"]
    .str.split("[")
    .str[1]
    .str.split("]")
    .str[0]
    .str.split("engine_x")
    .str[1]
    .str[1:]
)
df["test_fork"] = df["test_title"].str.split("fork_").str[1].str.split("-").str[0]
# Parse opcodes
df["test_opcode"] = df["test_params"].str.extract(r"(?:opcode_|op_)([^-]+)")
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_modexp", "MODEXP", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("selfdestruct"), "SELFDESTRUCT", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldatacopy", "CALLDATACOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_mcopy", "MCOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_codecopy", "CODECOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_returndatacopy", "RETURNDATACOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("returndatasize"), "RETURNDATASIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("extcodecopy"), "EXTCODECOPY", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldataload", "CALLDATALOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_keccak", "KECCAK", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_shifts",
    df["test_params"].str[-3:],
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_selfbalance", "SELFBALANCE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_msize", "MSIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_jumpdests", "JUMPDEST", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("jumpi"), "JUMPI", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_jumps", "JUMP", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_tstore", "TSTORE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_tload", "TLOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SSTORE"), "SSTORE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SLOAD"), "SLOAD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_log_opcodes", "LOG", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_calldatasize", "CALLDATASIZE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_blobhash", "BLOBHASH", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_blockhash", "BLOCKHASH", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_worst_callvalue", "CALLVALUE", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bn128_add"), "ecAdd", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bn128_mul"), "ecMul", df["test_opcode"]
)
df["test_opcode"] = np.where(
    (df["test_params"].str.contains("bn128"))
    & (df["test_params"].str.contains("pairing")),
    "ecPairing",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"] == "test_amortized_bn128_pairings", "ecPairing", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("ec_pairing"), "ecPairing", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("point_evaluation"),
    "point evaluation",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("blake2f"), "blake2f", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("ecrecover"), "ecRecover", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("SHA2-256"), "SHA2-256", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("RIPEMD-160"), "RIPEMD-160", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("IDENTITY"), "identity", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g1add"), "BLS12_G1ADD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g1msm"), "BLS12_G1MSM", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g2add"), "BLS12_G2ADD", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_g2msm"), "BLS12_G2MSM", df["test_opcode"]
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_pairing_check"),
    "BLS12_PAIRING_CHECK",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_fp_to_g1"),
    "BLS12_MAP_FP_TO_G1",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_params"].str.contains("bls12_fp_to_g2"),
    "BLS12_MAP_FP2_TO_G2",
    df["test_opcode"],
)
df["test_opcode"] = np.where(
    df["test_name"].str.contains("jumpdest_analysis"),
    "JUMPDEST",
    df["test_opcode"],
)
# Drop Nas and unnecessary column
df = df.drop(columns=['test_title'])
df = df.dropna()
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 20320 entries, 10198 to 13196
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client       20320 non-null  object 
 1   mgas/s       20320 non-null  float64
 2   test_file    20320 non-null  object 
 3   test_name    20320 non-null  object 
 4   test_params  20320 non-null  object 
 5   test_fork    20320 non-null  object 
 6   test_opcode  20320 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.2+ MB


In [95]:
worse_df = (
    df[df["mgas/s"]>0] # ignore zeros
    .groupby("test_opcode")
    ["mgas/s"]
    .quantile(0.01)
    .reset_index()
)
worse_df

,test_opcode,mgas/s
0,ADD,127.780
1,ADDMOD,78.000
2,ADDRESS,70.151
3,AND,125.390
4,BALANCE,223.770
...,...,...
153,ecMul,59.916
154,ecPairing,57.700
155,ecRecover,59.529
156,identity,86.876


## Underpriced opcodes

#### 100M gas limit -> 35M gas/second

In [96]:
filtered_df = worse_df[worse_df["mgas/s"]<=60].sort_values(by="mgas/s")
filtered_df["multiplier"] = round(100 / filtered_df["mgas/s"], 2)
filtered_df

,test_opcode,mgas/s,multiplier
17,CALL,8.4309,11.86
30,DELEGATECALL,8.6509,11.56
51,EXTCODEHASH,8.7647,11.41
129,STATICCALL,8.7847,11.38
67,MODEXP,11.4000,8.77
50,EXTCODECOPY,13.9090,7.19
18,CALLCODE,22.4540,4.45
157,point evaluation,31.9000,3.13
72,MULMOD,42.6590,2.34
151,blake2f,47.9560,2.09


In [97]:
len(filtered_df)

18

In [98]:
filtered_df = worse_df[worse_df["mgas/s"]>200].sort_values(by="mgas/s")
filtered_df["multiplier"] = round(100 / filtered_df["mgas/s"], 2)
filtered_df

,test_opcode,mgas/s,multiplier
137,SWAP15,200.12,0.50
136,SWAP14,205.17,0.49
138,SWAP16,205.95,0.49
144,SWAP7,206.95,0.48
135,SWAP13,207.56,0.48
131,SWAP1,207.56,0.48
142,SWAP5,208.00,0.48
134,SWAP12,208.17,0.48
132,SWAP10,208.39,0.48
145,SWAP8,211.00,0.47
